In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import ssl
import os
import re

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [2]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [4]:


def get_html_from_url(url):
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver') 
  browser.set_page_load_timeout(30) 
  browser.get(url)
   
  last_height = browser.execute_script('return document.body.scrollHeight')

  flag=1

  while flag==1:
    
    try:
       browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infine_scroll(last_height))
       last_height = new_height

    except:
        print("End of page reached")
        flag = 0
  html = browser.page_source
  return html

In [5]:
#helper function get first number in string

def str_first_num(s):
    first_num = ''
    flag_digit=False
    for el in s: 
        if el.isdigit():
            first_num += el
            flag_digit = True
        else:
            if flag_digit:
                break
    #If there are no upvotes replace '' with 0
    if first_num == '':
        first_num = 0
    return first_num
    

In [6]:
def unpack_list(li):
    unpacked_list = []
    for el in li:
        if isinstance(el, list):
            unpacked_list.extend([*el])
        elif isinstance(el, str):
            unpacked_list.extend([el])
    return unpacked_list


In [7]:
# a helper function that removes unwanted words/sections like url links
def strip_rgx_words(inp, regex_ = ['image','url']):
    #remove urls or images
    rgx_word = None
    for rgx in regex_:
      if rgx in inp:
          rgx_word = rgx
          break
    if rgx_word is not None:
      out = inp.split(rgx_word)[0]
    else:
      out =inp
    return out

In [8]:
#read question urls from csv file
src_path = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\bert_outcome_analysis\second_iteration\links_for_second_quora_scrap.csv"
df = pd.read_csv(src_path)

In [9]:
#get list of urls
url_list = list(df['0'].values)

In [10]:
len(url_list)


2467

In [11]:
#remove duplicate url
new_url_list = []
for url in url_list:
    if url not in new_url_list:
        new_url_list.append(url)

In [12]:
len(new_url_list)

1582

In [13]:
url_list = new_url_list

In [14]:
#for removing rows with low upvotes
def row_index_with_low_upvotes(l):
    to_remove = []
    for i,element in enumerate(l):
        if 'K' in element or 'M' in element:
            continue
        else:
            if int(element)<500:
                to_remove.append(i)
    return to_remove

#get followers count based on names
def get_person_followers_count(names):
    followers_count = []
    for name in names:
        name = name.replace(' ', '-')
        url_name = 'https://www.quora.com/profile/' + name
        browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver') 
        browser.set_page_load_timeout(30) 
        browser.get(url_name)
        name_html = browser.page_source
        name_soup = BeautifulSoup(name_html)
        name_followers = name_soup.find('div', {'class':"q-text qu-dynamicFontSize--small qu-color--gray"})
        try:
            name_followers = name_followers.find_next(text=True).strip()
        except:
            name_followers = 'UNKNOWN'
        followers_count.append(name_followers)
    return followers_count
    
def fill_list_counts(all_inf):
    answers_upvotes = []
    for j in all_inf:
        if j != []:
            upvote = j[0].find('span', {'class':'q-text qu-whiteSpace--nowrap qu-display--inline-flex qu-alignItems--center qu-justifyContent--center'}).find_next(text=True).strip()
        else:
            upvote = '0'
        answers_upvotes.append(upvote)
    return answers_upvotes
                

def fill_list_str(inf):
    answers_upvotes = []
    for ans_inf in inf:
        if ans_inf[0] == []:
           val = 'UNKNOWN'
        else:
            val = ans_inf[0].find_next(text=True).strip()
        answers_upvotes.append(val)
    return answers_upvotes


def answers_into_df(url):
    '''input: url for scrapping answers to a particular question.
        out: df with answers for each url/question'''
    #request url html
    
    #create soup object with htmal parser
    page_html = get_html_from_url(url)
    soup = BeautifulSoup(page_html)

    boxes = soup.find_all('div', {'class':"q-box qu-hover--bg--darken"})
  
    answers = [box.find_all('p', {'class':"q-text qu-display--block qu-wordBreak--break-word qu-textAlign--start"}) for box in boxes]
    answers_text = []
    for ans in answers:
        #if there are more than one element in list unpack all into one list         
        if len(ans) > 1:
            ans_text_list = [a.find_next(text=True).strip() for a in ans] 
            ans_text_whole = ''.join(ans_text_list)
        elif isinstance(ans, list):
            if ans == []:
                ans_text_whole = 'UNKNOWN'
                continue
            try:
                ans_text_whole = ans[0].find_next(text=True).strip()
            except:
                ans_text_whole = ans[0]
        else:
            ans_text_whole = ans
        answers_text.append(ans_text_whole)
 
    #get a list of answer author upvotes and shares
    #answers_info = [box.find_all('button', {'role':'button'}) for box in boxes]
    answers_upvotes_info = [box.find_all('div', {
        'class':'q-text qu-overflow--hidden qu-display--inline-flex qu-ml--tiny qu-minHeight--20 qu-color--gray qu-minWidth--20'
        }) for box in boxes]
     
    #answers upvotes:
    answers_upvotes = fill_list_counts(answers_upvotes_info)
    
    #keep only boxes with high upvotes 
    idx_low_upvotes = row_index_with_low_upvotes(answers_upvotes)
    boxes = [box for i,box in enumerate(boxes) if i not in idx_low_upvotes]
    if boxes == []:
        return None

    answers_upvotes = [upvote for i,upvote in enumerate(answers_upvotes) if i not in idx_low_upvotes]

    names_info = [box.find_all('div', {'class':'q-inlineFlex qu-alignItems--center qu-wordBreak--break-word'}) for box in boxes]
    answers_name = fill_list_str(names_info)
    
    names_follower_counts = get_person_followers_count(answers_name)
      
    author_counts = [box.find_all('span', {'class':"q-text qu-bold"}) for box in boxes]

    author_answers_count = []
    answer_views = []
        
    for i, aut_co in enumerate(author_counts):
        try:
            ans_co = aut_co[0].find_next(text=True).strip()
            ans_view = aut_co[1].find_next(text=True).strip()
        except:
            ans_co = 0
            ans_view = 0
        author_answers_count.append(ans_co)
        answer_views.append(ans_view)
    #if no answers than continue
    columns = ["Answer",  "name", "followerCount", "upvoteCount", "answerCount", "answerViews"]

    #create df with nans to fill it later with values
    
    x_shape = (len(boxes), len(columns))
    x = np.tile(np.nan, x_shape)

   
    answers_df = pd.DataFrame(x, columns = columns)
    if answers_text == []:
        return None
    answers_df['Answer'] = pd.DataFrame(answers_text)   
    answers_df['name'] = pd.DataFrame(answers_name)
    answers_df['followerCount'] = pd.DataFrame(names_follower_counts)
    answers_df['upvoteCount'] = pd.DataFrame(answers_upvotes)
    answers_df['answerCount'] = pd.DataFrame(author_answers_count)
    answers_df['answerViews'] = pd.DataFrame(answer_views)

    return answers_df 

In [15]:
#show one example
#answers_into_df(new_url_list[3])

In [16]:
save_path = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\bert_outcome_analysis\second_iteration"

def save_df(df, q, i):
    questions = unpack_list(q)
    df['question'] = pd.DataFrame(questions)
    df.reset_index(inplace=True)
    df.set_index('index')
    df.drop('index', axis=1, inplace=True)
    
    final_path = os.path.join(save_path, "ET_RCS_20OCT22_quora_answers_emerging_2nd_iteration_iter"+str(i)+".csv")
    df.to_csv(final_path)

In [17]:
#columns = ["Answer", "upvoteCount", "answerCount", "followerCount", "name"]
columns = ["Answer", "followerCount", "name", "upvoteCount", "answerCount", "answerViews"]
df = pd.DataFrame(data = [], columns = columns)
questions = []
j = 761
for i, url in enumerate(url_list[j:]):
    print(j)
    temp_df = answers_into_df(url)
    if temp_df is None:
       temp_df = pd.DataFrame([], columns=columns)
    questions.append([url]*len(temp_df))
    
    df = df.append(temp_df)
    if j%5 == 0 or j==761:
        save_df(df, questions, j)
        df = pd.DataFrame(data = [], columns = columns)
        questions = []
    j+=1
#save last questions
save_df(df, questions, i)
df = pd.DataFrame(data = [], columns = columns)

753


C:\Users\Ravit\AppData\Local\Temp\ipykernel_17924\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached


C:\Users\Ravit\AppData\Local\Temp\ipykernel_17924\133922634.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')
C:\Users\Ravit\AppData\Local\Temp\ipykernel_17924\1575332992.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_17924\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


754
End of page reached


...

759
End of page reached


C:\Users\Ravit\AppData\Local\Temp\ipykernel_17924\133922634.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')
C:\Users\Ravit\AppData\Local\Temp\ipykernel_17924\1575332992.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_17924\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


760
End of page reached


C:\Users\Ravit\AppData\Local\Temp\ipykernel_17924\133922634.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')
C:\Users\Ravit\AppData\Local\Temp\ipykernel_17924\1575332992.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
C:\Users\Ravit\AppData\Local\Temp\ipykernel_17924\4126610301.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


761


NameError: name 'df' is not defined